In [1]:
import xgboost as xgb
import numpy as np
from  sklearn.model_selection import train_test_split,GridSearchCV,KFold
from sklearn.metrics import confusion_matrix,mean_squared_error
from sklearn.datasets import load_iris,load_digits,load_boston
from hyperopt import fmin, tpe, hp, partial

In [2]:
iris = load_iris()
x = iris['data']
y = iris['target']
x

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [3]:
x_train_all,x_predcit,y_train_all,y_predict = train_test_split(x,y,test_size=0.1,random_state=100)
x_train,x_test,y_train,y_test = train_test_split(x_train_all,y_train_all,test_size=0.2,random_state=100)

In [4]:
d_train = xgb.DMatrix(data=x_train,label=y_train)
d_test = xgb.DMatrix(data=x_test,label=y_test)
eval_list = [(d_train,'train'),(d_test,'test')]

In [5]:
# 自定义hyperopt的参数空间
space = {"max_depth": hp.randint("max_depth", 15),
         "n_estimators": hp.randint("n_estimators", 300),
         'learning_rate': hp.uniform('learning_rate', 1e-3, 5e-1),
         "subsample": hp.randint("subsample", 5),
         "min_child_weight": hp.randint("min_child_weight", 6),
         }

def argsDict_tranform(argsDict, isPrint=False):
    argsDict["max_depth"] = argsDict["max_depth"] + 5
    argsDict['n_estimators'] = argsDict['n_estimators'] + 150
    argsDict["learning_rate"] = argsDict["learning_rate"] * 0.02 + 0.05
    argsDict["subsample"] = argsDict["subsample"] * 0.1 + 0.5
    argsDict["min_child_weight"] = argsDict["min_child_weight"] + 1
    if isPrint:
        print(argsDict)
    else:
        pass

    return argsDict

In [6]:
def xgboost_factory(argsDict):
    argsDict = argsDict_tranform(argsDict)
    
    params = {'nthread': -1,  # 进程数
              'max_depth': argsDict['max_depth'],  # 最大深度
              'n_estimators': argsDict['n_estimators'],  # 树的数量
              'eta': argsDict['learning_rate'],  # 学习率
              'subsample': argsDict['subsample'],  # 采样数
              'min_child_weight': argsDict['min_child_weight'],  # 终点节点最小样本占比的和
              'objective': 'reg:squarederror',
              'gamma': 0,  # 是否后剪枝
              'colsample_bytree': 0.7,  # 样本列采样
              'alpha': 0,  # L1 正则化
              'lambda': 0,  # L2 正则化
              'scale_pos_weight': 0,  # 取值>0时,在数据不平衡时有助于收敛
              'seed': 100,  # 随机种子
              }
    params['eval_metric'] = ['rmse']

    xrf = xgb.train(params, d_train, params['n_estimators'], eval_list,early_stopping_rounds=100)

    return get_tranformer_score(xrf)

def get_tranformer_score(tranformer):
    
    xrf = tranformer
    dpredict = xgb.DMatrix(x_predcit)
    prediction = xrf.predict(dpredict, ntree_limit=xrf.best_ntree_limit)
  
    return mean_squared_error(y_predict, prediction)

In [10]:
tpe.suggest?

Signature:
tpe.suggest(
    new_ids,
    domain,
    trials,
    seed,
    prior_weight=1.0,
    n_startup_jobs=20,
    n_EI_candidates=24,
    gamma=0.25,
    verbose=True,
)
Docstring:
Given previous trials and the domain, suggest the best expected hp point
according to the TPE-EI algo


Args:
    prior_weight(
    n_startup_jobs:
    n_EI_candidates:
    gamma:
    verbose:

Returns:
File:      e:\anaconda\lib\site-packages\hyperopt\tpe.py
Type:      function


In [9]:
algo2 = tpe.suggest(n_startup_jobs=1)

TypeError: suggest() missing 4 required positional arguments: 'new_ids', 'domain', 'trials', and 'seed'

In [13]:
type(tpe.suggest)

function

In [7]:
algo = partial(tpe.suggest, n_startup_jobs=1)
best1 = fmin(xgboost_factory, space, algo=algo, max_evals=20, pass_expr_memo_ctrl=None)

[13:33:20] WARNING: ..\src\learner.cc:576:            
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-rmse:0.94430	test-rmse:0.65768              
[1]	train-rmse:0.89683	test-rmse:0.62770              
[2]	train-rmse:0.85291	test-rmse:0.60144              
[3]	train-rmse:0.81239	test-rmse:0.57875              
[4]	train-rmse:0.77087	test-rmse:0.54583              
[5]	train-rmse:0.73572	test-rmse:0.52749              
[6]	train-rmse:0.70571	test-rmse:0.51734              
[7]	train-rmse:0.67557	test-rmse:0.49502              
[8]	train-rmse:0.64834	test-rmse:0.48250              
[9]	train-rmse:0.62380	test-rmse:0.47040              
[10]	train-rmse:0.59864	test-rmse:0.45273             
[11]	train-rmse:0.578

E:\anaconda\lib\site-packages\xgboost\core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(



[24]	train-rmse:0.44466	test-rmse:0.39125
[25]	train-rmse:0.44266	test-rmse:0.39277                                        
[26]	train-rmse:0.43886	test-rmse:0.39169                                        
[27]	train-rmse:0.43590	test-rmse:0.39129                                        
[28]	train-rmse:0.43522	test-rmse:0.39345                                        
[29]	train-rmse:0.43560	test-rmse:0.39252                                        
[30]	train-rmse:0.43598	test-rmse:0.39529                                        
[31]	train-rmse:0.43281	test-rmse:0.39156                                        
[32]	train-rmse:0.43355	test-rmse:0.39340                                        
[33]	train-rmse:0.43544	test-rmse:0.39691                                        
[34]	train-rmse:0.43721	test-rmse:0.39961                                        
[35]	train-rmse:0.43870	test-rmse:0.40193                                        
[36]	train-rmse:0.43959	test-rmse:0.40382               

E:\anaconda\lib\site-packages\xgboost\core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(



[22]	train-rmse:0.42233	test-rmse:0.37487                                        
[23]	train-rmse:0.41417	test-rmse:0.36757                                        
[24]	train-rmse:0.41360	test-rmse:0.37100                                        
[25]	train-rmse:0.41358	test-rmse:0.37320                                        
[26]	train-rmse:0.41114	test-rmse:0.37214                                        
[27]	train-rmse:0.40916	test-rmse:0.37130                                        
[28]	train-rmse:0.41003	test-rmse:0.37451                                        
[29]	train-rmse:0.41157	test-rmse:0.37438                                        
[30]	train-rmse:0.41408	test-rmse:0.37825                                        
[31]	train-rmse:0.41091	test-rmse:0.37433                                        
[32]	train-rmse:0.41321	test-rmse:0.37738                                        
[33]	train-rmse:0.41616	test-rmse:0.38116                                        
[34]	train-rmse:

...

E:\anaconda\lib\site-packages\xgboost\core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(



[26]	train-rmse:0.39028	test-rmse:0.33264
[27]	train-rmse:0.38633	test-rmse:0.33165                                          
[28]	train-rmse:0.38429	test-rmse:0.33159                                          
[29]	train-rmse:0.38186	test-rmse:0.33170                                          
[30]	train-rmse:0.38112	test-rmse:0.33262                                          
[31]	train-rmse:0.38019	test-rmse:0.33304                                          
[32]	train-rmse:0.37990	test-rmse:0.33480                                          
[33]	train-rmse:0.38156	test-rmse:0.33800                                          
[34]	train-rmse:0.38212	test-rmse:0.33401                                          
[35]	train-rmse:0.38208	test-rmse:0.33080                                          
[36]	train-rmse:0.38208	test-rmse:0.33108                                          
[37]	train-rmse:0.38400	test-rmse:0.33369                                          
[38]	train-rmse:0.38752	test-rmse:

E:\anaconda\lib\site-packages\xgboost\core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(



[24]	train-rmse:0.39485	test-rmse:0.33243                                          
[25]	train-rmse:0.39089	test-rmse:0.33198                                          
[26]	train-rmse:0.38656	test-rmse:0.33067                                          
[27]	train-rmse:0.38303	test-rmse:0.32978                                          
[28]	train-rmse:0.38150	test-rmse:0.33000                                          
[29]	train-rmse:0.37943	test-rmse:0.33021                                          
[30]	train-rmse:0.37916	test-rmse:0.33139                                          
[31]	train-rmse:0.37861	test-rmse:0.33208                                          
[32]	train-rmse:0.37869	test-rmse:0.33396                                          
[33]	train-rmse:0.38082	test-rmse:0.33741                                          
[34]	train-rmse:0.38180	test-rmse:0.33361                                          
[35]	train-rmse:0.38210	test-rmse:0.33054                                   

E:\anaconda\lib\site-packages\xgboost\core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(



In [8]:
best2 = fmin(xgboost_factory, space, algo=algo2, max_evals=20, pass_expr_memo_ctrl=None)

NameError: name 'algo2' is not defined

In [17]:
best1

{'learning_rate': 0.032365999663194325,
 'max_depth': 1,
 'min_child_weight': 5,
 'n_estimators': 201,
 'subsample': 2}

In [18]:
best2

{'learning_rate': 0.38238857156396555,
 'max_depth': 14,
 'min_child_weight': 4,
 'n_estimators': 189,
 'subsample': 2}